In [1]:
import os
import firebase_admin
import time
from tqdm.auto import tqdm
from firebase_admin import credentials, firestore

if '.gitignore' not in os.listdir():
    os.chdir('..')

In [2]:
cred = credentials.Certificate('website-cine-e77fb4ab2924.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

### Analysing movies from offline-database (in 2022)

In [4]:
data = db.collection("offline-database").stream() 
data_per_date = dict()
for elem in data:
    elem = elem.to_dict()
    data_per_date[elem['date']] = elem

movie_stream = db.collection("movie-info").stream()
movie_info = dict()
for elem in movie_stream:
    elem = elem.to_dict()
    movie_info[elem['id']] = elem

In [3]:
print(1)

1


In [27]:
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
import os

def plot_decades(decades_played, setxticks=False, cast_int=True):
    plt.clf()
    plt.figure(figsize=(12, 6), dpi=80)
    _ = plt.gca()
    if cast_int:
        plt.bar(decades_played[:,0].astype(int), decades_played[:,1].astype(int), width=2.5)
    else:
        plt.bar(decades_played[:,0], decades_played[:,1].astype(int), width=2.5)
    
    if setxticks:
        _, _ = plt.xticks(decades_played[:,0].astype(int))
    plt.show()

def get_counters_movies_and_dates(data_per_date, movie_info):
    movies_played = Counter()
    years_played = Counter()
    decades_played = Counter()
    directors_played = Counter()
    data2022 = list()
    for date, data in data_per_date.items():
        movies_date = data["movies"]
        year_date = int(date.split("_")[0])
        if (year_date==2022):
            for mov in movies_date:
                for k, v in movies_date[mov].items():
                    for hour in v["showtimes"]:
                        year = mov.split("-")[-1]
                        if year != "None":
                            if (int(year)<=2018) and (int(year)>=1900):
                                decade = str(year)[:3]+"0"
                                director = movie_info[mov]["directors"]
                                movies_played.update([mov])
                                years_played.update([year])
                                decades_played.update([decade])
                                directors_played.update([director])
                                data2022.append([mov, date, hour, year, decade, director])
    return movies_played, years_played, decades_played, directors_played, data2022

movies_played, years_played, decades_played, directors_played, data2022 = \
    get_counters_movies_and_dates(data_per_date=data_per_date, movie_info=movie_info)

data2022 = pd.DataFrame (data2022, columns = ['movie', 'date', 'hour', 'year', 'decade', 'director'])

# print(directors_played.most_common(10))
# print(movies_played.most_common(10))

# movies_played = np.array(movies_played.most_common(100))
# plot_decades(movies_played, setxticks=False, cast_int=False)

# years_played = np.array(sorted(years_played.items()))
# plot_decades(years_played, setxticks=False)

# decades_played = np.array(sorted(decades_played.items()))
# plot_decades(decades_played, setxticks=True)
print(data2022)

                                movie        date       hour  year decade  \
0                      vie-belle-1946  2022_01_01      13.25  1946   1940   
1                      vie-belle-1946  2022_01_01       16.0  1946   1940   
2                      vie-belle-1946  2022_01_01      18.75  1946   1940   
3                      vie-belle-1946  2022_01_01      21.25  1946   1940   
4                     parias-vie-1916  2022_01_01       14.0  1916   1910   
...                               ...         ...        ...   ...    ...   
33328                     le-bal-1983  2022_12_31       17.0  1983   1980   
33329           l-hiver-feerique-2015  2022_12_31       16.0  2015   2010   
33330                  mood-love-2000  2022_12_31  21.833333  2000   2000   
33331  comedie-erotique-nuit-ete-1982  2022_12_31       14.0  1982   1980   
33332                paris-texas-1984  2022_12_31  21.083333  1984   1980   

                                                director  
0               